# Setup

- Guide: https://medium.com/towards-agi/how-to-use-vertex-ai-with-langchain-for-your-projects-ca7c1022a900
- will do: https://python.langchain.com/docs/tutorials/llm_chain/

In [53]:
%pwd

'/Users/picetrp/Documents/Learn/My_Self_Learn/llm_hub/get_started'

In [84]:
# setup langsmith

import getpass
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass

os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API key (optional): "
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='Enter your LangSmith Project Name (default = "default"): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"
# if "OPENAI_API_KEY" not in os.environ:
#     os.environ["OPENAI_API_KEY"] = getpass.getpass(
#         prompt="Enter your OpenAI API key (required if using OpenAI): "
#     )


In [85]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../serious-hold-453009-g1-eff08c861e11.json"

In [13]:
# Ensure your VertexAI credentials are configured

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash-001", model_provider="google_vertexai")

In [14]:
model

ChatVertexAI(project='serious-hold-453009-g1', model_name='gemini-2.0-flash-001', full_model_name='projects/serious-hold-453009-g1/locations/us-central1/publishers/google/models/gemini-2.0-flash-001', client_options=ClientOptions: {'api_endpoint': 'us-central1-aiplatform.googleapis.com', 'client_cert_source': None, 'client_encrypted_cert_source': None, 'quota_project_id': None, 'credentials_file': None, 'scopes': None, 'api_key': None, 'api_audience': None, 'universe_domain': None}, default_metadata=(), model_family=<GoogleModelFamily.GEMINI_ADVANCED: '2'>)

# Prompt templates

- learn from: https://learn.deeplearning.ai/courses/langchain/lesson/xf7wh/models,-prompts-and-parsers

### Examples

In [7]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate tfrom langchanihe following from English into Italian"),
    HumanMessage("hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!\n', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 13, 'candidates_token_count': 3, 'total_token_count': 16, 'prompt_tokens_details': [{'modality': 1, 'token_count': 13}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 3}], 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.004586287774145603, 'model_name': 'gemini-2.0-flash-001'}, id='run-e44a3661-7f03-4820-9673-09b357cf98b0-0', usage_metadata={'input_tokens': 13, 'output_tokens': 3, 'total_tokens': 16})

In [9]:
for token in model.stream(messages):
    print(token.content, end="|")

Ciao|!
|

In [10]:
from IPython.display import display
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)
display(prompt_template)

prompt = prompt_template.invoke({"language": "Thai", "text": "hi!"})

display(prompt)
prompt.to_messages()

ChatPromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='Translate the following from English into {language}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})])

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Thai', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})])

[SystemMessage(content='Translate the following from English into Thai', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})]

In [11]:
response = model.invoke(prompt)
print(response.content)
response

สวัสดี! (Sawasdee!)



AIMessage(content='สวัสดี! (Sawasdee!)\n', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 9, 'candidates_token_count': 10, 'total_token_count': 19, 'prompt_tokens_details': [{'modality': 1, 'token_count': 9}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 10}], 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.15732163190841675, 'model_name': 'gemini-2.0-flash-001'}, id='run-e4be650b-e02a-403e-9fb6-53196e4e5d75-0', usage_metadata={'input_tokens': 9, 'output_tokens': 10, 'total_tokens': 19})

### Try

In [18]:
from langchain_core.prompts import ChatPromptTemplate

# zero-shot COT
template_string = """I went to the market and bought {n_apples} apples. \
I gave 2 apples to the neighbor and 2 to the repairman. \
I then went and bought 5 more apples and ate 1. \
How many apples did I remain with?
Let's think step by step.
"""

prompt_template = ChatPromptTemplate.from_template(template_string)
prompt = prompt_template.invoke({"n_apples": 201}) # => ChatPromptValue
# prompt = prompt_template.format_messages(n_apples=7) # => List[HumanMessage]
display(prompt)

response = model.invoke(prompt)
display(response)

ChatPromptValue(messages=[HumanMessage(content="I went to the market and bought 201 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1. How many apples did I remain with?\nLet's think step by step.\n", additional_kwargs={}, response_metadata={})])

AIMessage(content='1. Start with 201 apples.\n2. Give away 2 apples to the neighbor: 201 - 2 = 199 apples.\n3. Give away 2 apples to the repairman: 199 - 2 = 197 apples.\n4. Buy 5 more apples: 197 + 5 = 202 apples.\n5. Eat 1 apple: 202 - 1 = 201 apples.\n\nSo, you remained with 201 apples.\n', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 61, 'candidates_token_count': 121, 'total_token_count': 182, 'prompt_tokens_details': [{'modality': 1, 'token_count': 61}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 121}], 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.06128485734797706, 'model_name': 'gemini-2.0-flash-001'}, id='run-10fbaedd-3a45-463e-848c-fb26f519668e-0', usage_metadata={'input_tokens': 61, 'output_tokens': 121, 'total_tokens': 182})

In [22]:
print(response.content)

1. Start with 201 apples.
2. Give away 2 apples to the neighbor: 201 - 2 = 199 apples.
3. Give away 2 apples to the repairman: 199 - 2 = 197 apples.
4. Buy 5 more apples: 197 + 5 = 202 apples.
5. Eat 1 apple: 202 - 1 = 201 apples.

So, you remained with 201 apples.



# Output Parser

- used our openai key

### Example

In [31]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

# read local .env file
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

# initialize openai chat model - 2 ways
# from langchain.chat_models import init_chat_model
# chat = init_chat_model("gpt-4o-mini", model_provider="openai")

from langchain_openai import ChatOpenAI
chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # api_key="...",  # if you prefer to pass api key in directly instaed of using env vars
    # base_url="...",
    # organization="...",
    # other params...
)

In [33]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
messages = prompt_template.format_messages(text=customer_review)
response = chat(messages)
print(response.content)

/var/folders/2_/7_kx30fj0tgb803zjjs198_40000gn/T/ipykernel_71384/2691146417.py:37: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chat(messages)


```json
{
  "gift": true,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there", "I think it's worth it for the extra features"]
}
```


In [38]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [43]:
output_parser

StructuredOutputParser(response_schemas=[ResponseSchema(name='gift', description='Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.', type='string'), ResponseSchema(name='delivery_days', description='How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.', type='string'), ResponseSchema(name='price_value', description='Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.', type='string')])

In [40]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)
messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)
response = chat(messages)
print(response.content)

```json
{
	"gift": "True",
	"delivery_days": "2",
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


In [42]:
output_dict = output_parser.parse(response.content)
print(output_dict), type(output_dict)

{'gift': 'True', 'delivery_days': '2', 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}


(None, dict)

### Try

In [47]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

comment = """Okay for everyone wondering about Snow White’s hair here’s why it fails. \
Her hair is NOT A BOB the original Snow White had what was called a French Roll, an incredibly difficult hair style that requires A LOT of hair (you need at LEAST a foot of hair to pull it off and the more hair the easier it’s to do). \
Most people can’t even do a French Roll anymore without a wig because DRUMROLL PLEASE to keep a French roll in place? You have to Sew it into fabric. \
Yeah, that “bow” that snow has is actually what her hair is sewed to and wrapped around then tied at the top of her head. \
That’s why her hair is so thick at the end. French Rolls take hours to do and typically would be kept in the hair for over a week (no one wanted to undo it and redo it every day)"""

review_template = """\
For the following text, extract the following information:

sentiment: Is the comment positive or negative towards the subject? Answer Positive if yes, Negative if no. Answer Unknown if you are not sure.
subject: What person do the comment mainly talking about? If this information is not found, output -1.
reason: Extract any related sentences that is explained the sentiment of the comment. Output them as a comma separated Python list.

Format the output as JSON with the following keys:
sentiment: (str)
subject: (str)
reason: List[str]

text: {text}

{format_instructions}
"""

sentiment = ResponseSchema(name="sentiment", description="Was the comment shows a sign of positive or negative towards the subject? Answer True if yes, False if not or unknown if you are not sure.")
subject = ResponseSchema(name="subject", description="What person do the comment mainly talking about? If this information is not found, output -1.")
reason = ResponseSchema(name="reason", description="Extract any related sentences that is explained the sentiment of the comment. Output them as a comma separated Python list.")

response_schemas = [sentiment, subject, reason]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
display(format_instructions)
print()

prompt = ChatPromptTemplate.from_template(template=review_template)
messages = prompt.format_messages(text=comment, 
                                format_instructions=format_instructions)
response = chat(messages)
response_dict = output_parser.parse(response.content)
print(response_dict)

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"sentiment": string  // Was the comment shows a sign of positive or negative towards the subject? Answer True if yes, False if not or unknown if you are not sure.\n\t"subject": string  // What person do the comment mainly talking about? If this information is not found, output -1.\n\t"reason": string  // Extract any related sentences that is explained the sentiment of the comment. Output them as a comma separated Python list.\n}\n```'


{'sentiment': 'Negative', 'subject': 'Snow White', 'reason': ['her hair is NOT A BOB', 'the original Snow White had what was called a French Roll', 'an incredibly difficult hair style that requires A LOT of hair', 'Most people can’t even do a French Roll anymore without a wig', 'to keep a French roll in place? You have to Sew it into fabric', "that 'bow' that snow has is actually what her hair is sewed to and wrapped around then tied at the top of her head", 'that’s why her hair is so thick at the end', 'French Rolls take hours to do and typically would be kept in the hair for over a week']}


In [49]:
comment = """I watched it yesterday. She's the best part of the movie, idc what anyone says but she really carried the movie."""
prompt = ChatPromptTemplate.from_template(template=review_template)
messages = prompt.format_messages(text=comment, 
                                format_instructions=format_instructions)

# try change temperature
chat = ChatOpenAI(model="gpt-4o-mini", temperature=1.0)
response = chat(messages)
response_dict = output_parser.parse(response.content)
print(response_dict)

{'sentiment': 'Positive', 'subject': 'She', 'reason': ["She's the best part of the movie", 'she really carried the movie']}


# Conversational Memory (langchain 0.2 - deprecated)
- Memory classes (langchain 0.2 - deprecated) But still worth understanding: `**ConversationBufferMemory**`, `**ConversationBufferWindowMemory**`, `**ConversationTokenBufferMemory**`, `**ConversationSummaryMemory**`
- Can read at: https://www.aurelio.ai/learn/langchain-conversational-memory or ask gemini

## ConversationBufferMemory

* **Core Idea:** This is the simplest form of memory. It stores the entire conversation history as a single string or a list of messages. Every interaction (user input and bot response) is appended to the buffer.

* **Example:**
    ```
    User: Hello!
    Bot: Hi there.
    User: What's the capital of Thailand?
    Bot: The capital of Thailand is Bangkok.
    ```
    **Memory Contents:** `"Human: Hello!\nAI: Hi there.\nHuman: What's the capital of Thailand?\nAI: The capital of Thailand is Bangkok."`

## ConversationBufferWindowMemory

* **Core Idea:** Keeps only the last `k` interactions (user input and bot response).

* **Example (with `k=2`):**
    ```
    User: What's the weather like in Bangkok?
    Bot: It's currently hot and humid.
    User: And the temperature?
    Bot: Around 32°C.
    ```
    **Memory Contents:** `"Human: And the temperature?\nAI: Around 32°C."` (The first two turns are discarded).

## ConversationTokenBufferMemory

* **Core Idea:** Stores recent interactions, limited by a maximum number of tokens.

* **Example (with `max_token_limit=10`, simplified token count):**
    ```
    User: Hi (1 token)
    Bot: Greetings (2 tokens)
    User: Tell me something interesting about Bangkok. (7 tokens)
    Bot: Bangkok is known for its vibrant street food. (7 tokens)
    ```
    **Memory Contents (might discard "Hi" and "Greetings" to stay within limit):** `"Human: Tell me something interesting about Bangkok.\nAI: Bangkok is known for its vibrant street food."`

## ConversationSummaryMemory

* **Core Idea:** Maintains a summarized version of the conversation over time.

* **Example:**
    ```
    User: I'm planning a trip to Bangkok next month.
    Bot: That sounds exciting!
    User: What are some must-see attractions?
    Bot: You should definitely visit the Grand Palace and Wat Arun.
    ```
    **Memory Contents (evolving summary):**
    * **After first turn:** `"The user is planning a trip to Bangkok."`
    * **After second turn:** `"The user is planning a trip to Bangkok and is asking for must-see attractions."`
    * **After third turn:** `"The user is planning a trip to Bangkok and has been recommended the Grand Palace and Wat Arun."`

In [58]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

# Conversational Memory (langchain 0.3+)
- Can read at: https://www.aurelio.ai/learn/langchain-conversational-memory or ask gemini
- We'll work through each of these memory types in turn, and rewrite each one using the `RunnableWithMessageHistory` class.

In [55]:
# import os
# from getpass import getpass
# from langchain_openai import ChatOpenAI

# os.environ["OPENAI_API_KEY"] = os.environ["OPENAI_API_KEY"] or getpass("Enter your OpenAI API key: ")

# # For normal accurate responses
# llm = ChatOpenAI(temperature=0.0, model="gpt-4o-mini")

In [73]:
# Ensure your VertexAI credentials are configured

from langchain.chat_models import init_chat_model
llm = init_chat_model("gemini-2.0-flash-001", model_provider="google_vertexai", temperature=0.0)

### ConversationBufferMemory (Old)

In [71]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

In [66]:
memory.save_context(
    {"input": "Hi, my name is Pice"},  # user message
    {"output": "Hey Pice, what's up? I'm an AI model called Zeta."}  # AI response
)
memory.save_context(
    {"input": "I'm researching the different types of conversational memory."},  # user message
    {"output": "That's interesting, what are some examples?"}  # AI response
)
memory.save_context(
    {"input": "I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory."},  # user message
    {"output": "That's interesting, what's the difference?"}  # AI response
)
memory.save_context(
    {"input": "Buffer memory just stores the entire conversation, right?"},  # user message
    {"output": "That makes sense, what about ConversationBufferWindowMemory?"}  # AI response
)
memory.save_context(
    {"input": "Buffer window memory stores the last k messages, dropping the rest."},  # user message
    {"output": "Very cool!"}  # AI response
)


In [67]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi, my name is Pice', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey Pice, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}, response_metadata={}),
  HumanMessag

In [74]:
memory = ConversationBufferMemory(return_messages=True)

memory.chat_memory.add_user_message("Hi, my name is Pice")
memory.chat_memory.add_ai_message("Hey Pice, what's up? I'm an AI model called Zeta.")
memory.chat_memory.add_user_message("I'm researching the different types of conversational memory.")
memory.chat_memory.add_ai_message("That's interesting, what are some examples?")
memory.chat_memory.add_user_message("I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.")
memory.chat_memory.add_ai_message("That's interesting, what's the difference?")
memory.chat_memory.add_user_message("Buffer memory just stores the entire conversation, right?")
memory.chat_memory.add_ai_message("That makes sense, what about ConversationBufferWindowMemory?")
memory.chat_memory.add_user_message("Buffer window memory stores the last k messages, dropping the rest.")
memory.chat_memory.add_ai_message("Very cool!")

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi, my name is Pice', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey Pice, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}, response_metadata={}),
  HumanMessag

In [75]:
from langchain.chains import ConversationChain

chain = ConversationChain(
    llm=llm, 
    memory=memory,
    verbose=True
)

In [77]:
response = chain.invoke({"input": "what is my name again?"})
response



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='Hi, my name is Pice', additional_kwargs={}, response_metadata={}), AIMessage(content="Hey Pice, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}), HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}), HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what's the difference?", additional_kwar

{'input': 'what is my name again?',
 'history': [HumanMessage(content='Hi, my name is Pice', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey Pice, what's up? I'm an AI model called Zeta.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I've been looking at ConversationBufferMemory and ConversationBufferWindowMemory.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's interesting, what's the difference?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer memory just stores the entire conversation, right?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='That makes sense, what about ConversationBufferWindowMemory?', additional_kwargs={}, 

In [83]:
display(response['response']) # cheeky

"I already told you, your name is Pice! I'm good at remembering things, so don't worry, I won't forget. 😉\n"

### ConversationBufferMemory (with RunnableWithMessageHistory)

In [88]:
from langchain.prompts import (
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    ChatPromptTemplate
)

system_prompt = "You are a helpful assistant called Begita."

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{query}"),
])

prompt_template

ChatPromptTemplate(input_variables=['history', 'query'], input_types={'history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_c

In [96]:
pipeline = prompt_template | llm

In [99]:
from langchain_core.chat_history import InMemoryChatMessageHistory

chat_map = {}
def get_chat_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in chat_map:
        # if session ID doesn't exist, create a new chat history
        chat_map[session_id] = InMemoryChatMessageHistory()
    return chat_map[session_id]

In [100]:
from langchain_core.runnables.history import RunnableWithMessageHistory

pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history"
)

In [102]:
pipeline_with_history.invoke(
    {"query": "Hi, my name is Pice"},
    config={"session_id": "id_123"}
)

AIMessage(content="Hello Pice, it's nice to meet you! How can I help you today?\n", additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 16, 'candidates_token_count': 20, 'total_token_count': 36, 'prompt_tokens_details': [{'modality': 1, 'token_count': 16}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 20}], 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.060756373405456546, 'model_name': 'gemini-2.0-flash-001'}, id='run-812a6a27-b522-455c-9286-687e6c33253e-0', usage_metadata={'input_tokens': 16, 'output_tokens': 20, 'total_tokens': 36})

In [103]:
pipeline_with_history.invoke(
    {"query": "What is my name again?"},
    config={"session_id": "id_123"}
)

AIMessage(content='Your name is Pice.\n', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 42, 'candidates_token_count': 7, 'total_token_count': 49, 'prompt_tokens_details': [{'modality': 1, 'token_count': 42}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 7}], 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.003015541338494846, 'model_name': 'gemini-2.0-flash-001'}, id='run-54264832-a470-46e5-9a14-cdb17d477d33-0', usage_metadata={'input_tokens': 42, 'output_tokens': 7, 'total_tokens': 49})

In [104]:
pipeline_with_history.invoke(
    {"query": "What is my name again?"},
    config={"session_id": "id_123"}
)

AIMessage(content='As I recall, your name is Pice.\n', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 55, 'candidates_token_count': 11, 'total_token_count': 66, 'prompt_tokens_details': [{'modality': 1, 'token_count': 55}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 11}], 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.12572927908463913, 'model_name': 'gemini-2.0-flash-001'}, id='run-656412be-f28d-47e0-add7-0d5cc187c5de-0', usage_metadata={'input_tokens': 55, 'output_tokens': 11, 'total_tokens': 66})

In [113]:
# Accessing the chat history for session "id_123"
history = get_chat_history("id_123")

# Print the entire history
for message in history.messages:
    print(message.content)

Hi, my name is Pice
Hello Pice, it's nice to meet you! How can I help you today?

What is my name again?
Your name is Pice.

What is my name again?
As I recall, your name is Pice.



### ConversationBufferWindowMemory with RunnableWithMessageHistory

# Build a Chatbot

# Semantic Search in PDF